# Group consistency check

insert anchors into the two patterns and compare their performance with (without) anchors

In [1]:
import pandas as pd 
import json 
import copy
import re 
import math
from pathlib import Path
import os, sys

from collections import Counter, defaultdict, OrderedDict
from copy import deepcopy
import pathlib

pd.set_option('display.max_columns',100)
pd.set_option('display.max_colwidth',500)
from tqdm import tqdm
tqdm.pandas()
import string
from inflection import pluralize, singularize
from util_wordnet import get_sister_terms
from transformers import pipeline

import spacy
en = spacy.load('en_core_web_sm')
STOP_WORDS = en.Defaults.stop_words

from IPython.display import display
from df_to_latex import DataFrame2Latex

ModuleNotFoundError: No module named 'util_wordnet'

In [3]:
# -*- coding: utf-8 -*-
"""## Import Modules"""

import os, sys
import json 
import math
import pandas as pd 
from tqdm import tqdm 
import re 
from sklearn.metrics import accuracy_score
from collections import  defaultdict, Counter, OrderedDict
from transformers import pipeline
import numpy as np 
import copy
import math
import configparser
import yaml 
from IPython.display import display
##### spacy 
import spacy
en = spacy.load('en_core_web_sm')
STOP_WORDS = en.Defaults.stop_words

import re
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from utils_wordnet import get_wordnet_shortest_path_score_between, get_wordnet_avg_path_score_between_sub_and_anchors, get_wordnet_shortest_path_length_between
from utils_concept_positioning_test import concpet_positioning_test, get_revserse_prompt
from inflection import singularize, pluralize 

"""Anchored_Prompts.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1CnAjbN-8oYPkxM0q4NT1uXVf5KV-GxZ8
"""

# !pip install transformers
# !pip install spacy



"""## Patterns"""




relations = [
    {
        "relation": "IsA",
        "def_sap": ["A [X] is a type of [Y] ."],
        "def_dap": ["A [X] or [Z] is a type of [Y] ."],
        "lsp_sap": ["such [Y] as [X]"],
        "lsp_dap": ["such [Y] as [X] or [Z]"],
        "use_dap": True,
        "anchor_target": "[X]",
        "anchor_lsp_sap": ["Such as [X] and [MASK] ."],
        # "dap_x": "A [X] and [Z] are a type of [Y] .",
        # "dap_y": "",
    },
]


def get_relation_templates(relations, model, anchor_type, def_sap_patterns, def_dap_patterns, lsp_sap_patterns, lsp_dap_patterns): 
    '''
    generate the relation tempaltes based on model and anchor_type 

    '''
    # let's assumet the anchor concept is [Z] 
    relation_to_template = defaultdict() 
    for relation_initial in relations:
        # print(relation.keys())
        relation_name = relation_initial['relation'] 
        # relation_to_template[key] = defaultdict()
        relation = copy.deepcopy(relation_initial)
        # anchor_target  = relation["anchor_target"]

        relation['anchor_lsp_sap']  = lsp_sap_patterns[anchor_type]
        relation['def_sap']  = def_sap_patterns[relation_name]
        relation['def_dap']  = def_dap_patterns[relation_name]

        relation['lsp_sap']  = lsp_sap_patterns[relation_name]
        relation['lsp_dap']  = lsp_dap_patterns[relation_name]

        if relation["anchor_target"] == '[X]':
            for pattern in ['def_sap', 'lsp_sap', 'def_dap', 'lsp_dap', 'anchor_lsp_sap']:
                if 'roberta' in model:
                    relation[pattern] = [item.replace("[Y]", "<mask>") for item in relation[pattern]]
                elif 'bert' in model: 
                    relation[pattern] = [item.replace("[Y]", "[MASK]") for item in relation[pattern]]
            
        # save_dict_to_json(new_relations, 'log/relation_sap_dap.json')
        # if debug:
            # print( json.dumps(relation_to_template, indent=4)
        relation_to_template[relation_name] = relation 
    return relation_to_template


"""## Helper Functions"""


def custom_tokenizer(nlp):
    infix_re = re.compile(r'''[.\,\?\:\;\...\‘\’\`\“\”\"\'~]''')
    prefix_re = compile_prefix_regex(nlp.Defaults.prefixes)
    suffix_re = compile_suffix_regex(nlp.Defaults.suffixes)

    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                token_match=None)

nlp = spacy.load('en_core_web_sm')
nlp.tokenizer = custom_tokenizer(nlp)

pd.options.display.max_columns = 50
pd.set_option('display.width', 1000)

"""## Helper Functions"""

def save_dict_to_json(examples, output_path):
    ''' 
    save a list of dicts into otuput_path, orient='records' (each line is a dict) 
    examples: a list of dicts
    output_path: 
    '''
    # if not os.path.exists(output_path):
        # os.path.makedirs(output_path)

    with open(output_path, 'w') as fout:
        for example in examples:
            json.dump(example, fout)
            fout.write("\n")
        print(f"save {output_path} with {len(examples)} lines")

"""## Load Data"""

def tokenize_sentence(sentence):
  sentence = sentence.replace("\"", "").lower()
  tokens = [token.orth_ for token in nlp(sentence)]
  return tokens

def locate_sub_obj_position(ent, sentence, index_not_in) :
  ''' 
  function: find the index of ent in a sentence, the result will be used to filter instances whose ent cannot be find at their sentences
  args: 
    sentence: the sentnces to mask, could be the string or a list of tokens 
    ent: the ent to be found (sub_label) 
    index_not_in: the default index for failed instances (an ent not in a sentence)
  ''' 

  if isinstance(sentence, list):
    if ent not in sentence:
      return index_not_in
    return sentence.index(ent)  
  else:
    sentence = copy.deepcopy(sentence).lower()
    if isinstance(sentence, str):
      try:
        index = sentence.index(ent)
        return  index 
      except: 
        print(f"NOT FOUND sub_label: {ent} -> in sentence: {sentence}")
        return index_not_in
      
        print(ent, sentence)
        return index_not_in

def remove_noisy_test_data(df):
  ''' 
  relation="hasproperty"
  why? some data points don't belong to this relation types 
  case1., sub_label=number, such as "10 is ten."  We don't say ten is the property of 10
  case2, sub_label = 'person_name' and obj_label = 'nuts;, such as ""Andrew is [MASK].", [MASK]=nuts
  '''
  sub_labels_to_exclude = ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '30', '5', '50', '60', '7', '70', '70s', '80', '9', '90']
  obj_labels_to_exclude  = ['nuts']
  df = df.query(f"sub_label not in {sub_labels_to_exclude}")
  df = df.query(f"sub_label not in {obj_labels_to_exclude}")
  return  df.reset_index(drop=True)

# def exist_
  
def load_data(filepath, clean_test=True, tokenize=False):
  '''
  return the cleaned data
  args:
    tokenize: if True: the maksed_sentences will be tokenzied (this is slwoers); 
            otherwise, we use the string match to filter the failed sentences
    clean_test: default is True. We filter out some noisy samples spoted by huamns 
               Note that this is relation specific 

  '''
  index_not_in = 10000

  with open(filepath, 'r', encoding='utf-8') as fin:
    data = fin.readlines()
    data = [eval(x) for x in data]
    df = pd.DataFrame(data)

#     df['obj_label'] = df['obj_label'].apply(lambda x: [x] if isinstance(x, str) else x)

#   if tokenize:
#     df['masked_sentence_tokens'] = df['masked_sentences'].apply(lambda x: tokenize_sentence(x[0]))
#     df['sub_position'] = df[['sub_label', 'masked_sentence_tokens']].apply(lambda x: locate_sub_obj_position(x[0], x[1], index_not_in=index_not_in), axis=1)

#   if clean_test: 
#     df = remove_noisy_test_data(df)
#     df['sub_position'] = df[['sub_label', 'masked_sentences']].apply(lambda x: locate_sub_obj_position(x[0], x[1][0], index_not_in), axis=1)
#     df = df.query(f"sub_position !={index_not_in}") #.reset_index() #cue can not be matched in the sentence

  print(f"#Test_instances: {len(df.index)}")
  return df.reset_index(drop=True)


def _get_article(word):
    
    if word[0] in ['a', 'e', 'i', 'o', 'u']:
        return 'an'.capitalize()
    return 'a'.capitalize()

def check_articles(sub_label, pattern):
    '''
    select suitable article templates for a given sub_label 
    '''
    first_word = pattern.split()[0]
    if first_word == sub_label:  
        return pattern.capitalize()

    elif first_word != sub_label: 
        sub_article = _get_article(sub_label)
        return pattern  if sub_article == first_word  else 'DISCARD'


    


def get_masked_data(filepath, relation, relation_to_template, model, singular_pattern_flag, debug=False, clean_test=True):
  df = load_data(filepath, clean_test=clean_test)
  if debug:
      df = df.sample(200)

  name_to_patterns = {'def_sap': def_sap_patterns, 'def_dap': def_dap_patterns, 'lsp_sap': lsp_sap_patterns, 'lsp_dap': lsp_dap_patterns, }
                  
  #   replace [X] by mask
  if 'roberta' in model :
    df['masked_sentences'] = df['masked_sentences'].apply(lambda x: [sentence.replace("[MASK]", "<mask>") for sentence in x])

  for pattern in ['def_sap', 'def_dap']:# , 'lsp_sap', 'anchor_lsp_sap']:
    # use check when sub_label are sigular, we nned to inderst an article before it
    if singular_pattern_flag:
      df[pattern] = df['sub_label'].apply(lambda x: [item.replace('[X]', x) for item in relation_to_template[pattern] if check_articles(x, item)!='DISCARD' ])
    else:
      df[pattern] = df['sub_label'].apply(lambda x: [item.replace('[X]', x) for item in relation_to_template[pattern] ])

  for pattern in ['lsp_sap', 'lsp_dap', 'anchor_lsp_sap']:
      df[pattern] = df['sub_label'].apply(lambda x: [item.replace('[X]', x) for item in relation_to_template[pattern]])

#   for pattern in ['def_dap', 'lsp_dap']:
    #   df[pattern] = df['sub_label'].apply(lambda x: [item.replace('[X]', x) for item in relation_to_template[pattern] if check_articles(x, item)!='DISCARD'])

  if debug:
    print(df.columns)
    print(f"{len(df.index)} instances")

  return df 

# if debug:
def test_load_data():
    for relation in ['IsA']: #['HasProperty']: # ['HasProperty', 'CapableOf', 'HasProperty', 'UsedFor', 'ReceivesAction']:
        print(f"relation: {relation}")
        filepath = f'./clsb/{relation}.jsonl'
        # df = load_data(filepath, clean_test=True, tokenize=False)
        relation_to_template  = get_relation_templates(relations, model, anchor_type='Coordinate')
        df = get_masked_data(filepath, relation, relation_to_template[relation], model)
        display(df.head())

"""## Define model and feed masked data into models"""

def get_unmasker(model, device, targets=None):
    if targets is None: 
        unmasker = pipeline('fill-mask', model=model, device=device)# 'bert-large-uncased') #initialize the masker
    else:
        unmasker = pipeline('fill-mask', model=model, device=device, targets=targets )# 'bert-large-uncased') #initialize the masker
    return unmasker


def get_target_vocab(file_path = "data/clsb/blank/vocab.txt"):
    '''
    Weir et al., (2020) used two variants of target vocabulary when predicting the possible properties 
    One of them is the SNES version, "comprising the set of human completion that fit the given prompt syntax for all concepts in the study"
    '''
    df_target_vocab = pd.read_csv(file_path, header=None) #.to_list()
    target_vocab = df_target_vocab.iloc[:,0].to_list() 
    return target_vocab



def develop_single_case():
    target_vocab = get_target_vocab(file_path = "data/clsb/blank/vocab.txt")
    unmasker = get_unmasker(model= 'bert-base-cased', targets= target_vocab )
    text = 'Everyone knows that a bear has [MASK] .'
    results = unmasker(text, top_k= len(target_vocab))

    for x in results[:10]:
        print(x)
    print("-"*80)

    unmasker = get_unmasker(model= 'bert-base-cased')
    text = 'Everyone knows that a bear has [MASK] .'
    results = unmasker(text, top_k= 10)
    for x in results[:10]:
        print(x)

def fill_mask_anchor(df, unmasker, anchor_col):
    outputs = defaultdict()
    outputs[anchor_col]  = [unmasker(x, top_k=top_k) for x in df[anchor_col]]

    return outputs

def fill_mask_obj(df, unmasker):
    outputs = defaultdict()
    # outputs['masked_sap'] = unmasker(df.masked_sap.to_list(), top_k=top_k, batch_size=100) 
    outputs['masked_sentences']  = [unmasker(x, top_k=top_k) for x in df['masked_sentences'].to_list()]

    outputs['def_sap']  = [unmasker(x, top_k=top_k) for x in df['def_sap'].to_list()]
    outputs['lsp_sap']  = [unmasker(x, top_k=top_k) for x in df['lsp_sap'].to_list()]
    # outputs['masked_sentences_obj_anchor'] = unmasker(df.masked_sentences_obj_anchor.to_list(), top_k=top_k, batch_size=100) 
    # outputs['masked_sentences'] = unmasker(df.masked_sentences.to_list(), top_k=top_k, batch_size=100)
    # outputs['masked_sentences']  = [unmasker(x, top_k=top_k) for x in tqdm(df.masked_sentences)]
    return outputs

"""## Cleaning outputs """

def aggregate_token_scores(input_word, token2probs, scorer, top_k, add_wordnet_path_score=False, sort_flag=True ):
    ''' 
    goal: we want the best scorer to consider:
        (1) frequency: a token that are elicited by multiple promptso
        (2) the probability: higher overall probability 
        (3)

    token2prob: dictionary mapping a token to a list of probs 
    anchor_anchor_scorer_list = ['freqProbSum', 'probMultiply', 'probMultiplyAvg', 'freqProbMultiply', 'freq', 'probSum', 'probAvg'] #TODO: rank based


    test case:
    token2probs = {'achieve': [0.2, 0.1, 0.03, 0.006], 'tried': [0.008, 0.006, 0.003, 0.001], 'perform':[0.08], 'prevent': [0.06], 'use': [0.02], 'accomplished': [0.1], 'produce':[0.06]}
    for scorer in  [ 'freqProbSum', 'probMultiply', 'probMultiplyAvg', 'freqProbMultiply' ]: #'freq', 'probSum', 'probAvg',
        token2prob = aggregate_token_scores(token2probs, scorer, top_k=7, sort_flag=True)
        print(scorer)
        print(f"\t{token2prob}" )
        print()

    '''
    token2prob = defaultdict()
    all_count = sum([len(item) for item in token2probs.values()])
    # print(json.dumps(token2probs, indent=4))
    for token, probs in token2probs.items(): #rank_score = w * p, w is the frequency weight, p is the probability
            count = len(probs)
            
            freq_weight = count/all_count
            
            new_score = 0 
            
            if scorer=='freq':
                new_score =  freq_weight 

            elif scorer=='probSum':
                new_score = sum(probs)

            elif scorer=='probAvg': #this ignore the frequency factor [not ideal]
                new_score = sum(probs)/ len(probs)

            elif scorer=='freqProbSum': #[close to ideal]
                new_score = freq_weight * sum(probs)
                # print(token, freq_weight,sum(probs), new_score )
            elif scorer=='probLogSum':
                probs_valid = [item for item in probs if item>0]
                if len(probs_valid )==0:
                    new_score= 0
                else:
                    new_score =  sum([math.log(item, 2) for item in probs_valid ])/len(probs_valid)
                    # new_score =  sum([math.log(item, 2) for item in probs if item>0])/len(probs)

            elif scorer=='freqProbLogSum': #[close to ideal, requires a token to be (1) frequent (2) high probs across prompts]
                probs_valid = [item for item in probs if item>0]
                if len(probs_valid )==0:
                    new_score= 0
                else:
                    new_score =   sum([math.log(item*freq_weight, 2) for item in probs_valid ])/len(probs_valid)
                    # new_score =   sum([math.log(item*freq_weight, 2) for item in probs if item>0])/len(probs)
                # print("token: {}, freq_weight: {}, logsum: {}, new_score: {}".format(token, freq_weight, sum([math.log(item, 2) for item in probs])/len(probs), new_score))
            # elif scorer=='probMultiply': #using prob multiply will penelize the tokens with high frequency
            #     new_score =  math.exp(sum([math.log(item, 2) for item in probs]))

            # 221114: add the WN path socre here to obtain reliable outputs 
            if add_wordnet_path_score:
                path_score = get_wordnet_shortest_path_score_between(input_word, token)
                #print(f"input_word: {input_word}", "token:", token )
                #print("path_score", path_score, type(path_score))
                #print("new_score", new_score, type(new_score))
                new_score += path_score  

            token2prob[token] = new_score
    
    # token2prob = sorted(token2prob.items(), key=lambda x: x[1], reverse=True )
    # if top_k is not None and isinstance(top_k, int):
        # token2prob = token2prob[:top_k] 
    # token2prob = dict(token2prob)
    return token2prob


def filter_outputs_with_probs(inputs, outputs, filter=True, return_probs=True, top_k=None, scorer='freqProbSum', filter_inputs=True, add_wordnet_path_score=False, add_cpt_score=False, cpt_unmasker=None, mask_string=None, cpt_only=False):
    '''
    inputs: the original inputs, for example [A] is a type of [B], A is the input
    outputs: the candidates returned by PTLMs

    filter: True 
        filter: non-alpha tokens); 

    top_k: take the top_k outputs. This is important when using multiple prompts for each sub 
    add_wordnet_path_score: add wordnet path score into the output scoring function 
    add_cpt_score: add concept-positioning test score into the output scoring function 

    '''
    anchor_list = []
    anchor_scores = [] 
        
    for input_word, top_outputs in zip(inputs, outputs):  #iterate through the samples (sub)
        filled_tokens  = defaultdict(int) #filter/accumulate predictions for each sample 
        filled_scores = defaultdict(list) #a list of token:[score1, score2, ...]   
        token2cpt  = defaultdict(list) #filter/accumulate predictions for each sample 

        if isinstance(top_outputs[0], list):
            flatten_output = [item for top_k_output  in top_outputs for item in top_k_output]
        else:
            flatten_output = [item for item  in top_outputs]
        # print("flatten_output", flatten_output)

        for i, output in enumerate(flatten_output):
            filled_token = output['token_str'].strip().lower()
            filled_score = output['score']
            if filter:
                #####Add conditions to filter unwanted ################
                # filter the repetation of a concept in the explanation. See the the following example
                # [MASK] is the capability to do a particular job . -> capacity 
                if not filled_token.isalpha(): continue
                if filled_token in STOP_WORDS: continue 
                if len(filled_token)<=1: continue 
                # if filter_inputs:
                if filled_token.strip().lower() in [re.sub("\s+", '', x) for x in input_word.split()]: continue #filter out the target in input  
                if filled_token.startswith("#"): continue
                #####Add conditions to filter unwanted ################
                
                filled_tokens[filled_token] +=1
                filled_scores[filled_token].append(filled_score)

                ##### add CPT test for filtering ################ 
                if add_cpt_score and cpt_unmasker!=None and mask_string!=None:
                    token2cpt[filled_token].append(concpet_positioning_test(stimulus_word=input_word,
                                                                            output= output, 
                                                                            unmasker=cpt_unmasker, 
                                                                            mask_string=mask_string))

                    # token2cpt_prompt[filled_token] =  get_revserse_prompt((stimulus_word=input_word,
                                                                            # output= output))
                ##### add CPT test for filtering ################ 
            else:
                filled_tokens[filled_token] +=1
                filled_scores[filled_token] += filled_score

        if len(filled_tokens) ==0: 
            filled_tokens={'MISSING':1}
            filled_scores['MISSING'] = [0]

        # feed the input into the agrregate _token_scores() so that we can calcuate the 
        token2probs = aggregate_token_scores(input_word, token2probs=filled_scores, scorer=scorer, top_k=top_k,  add_wordnet_path_score=add_wordnet_path_score, sort_flag=True)

        if filter and add_cpt_score:
            # extreme case: all anchors fail the CPT and return 0; the anchor selection would become random 
            # token2cpt[filled_token] = 
            token2cptsum = dict({token: sum(scores)/len(scores) for token, scores in token2cpt.items()})
            for token, probs in token2probs.items():
                token2probs[token] = probs + token2cptsum[token]
                # print(input_word, token, probs,token2probs[token] )
            
        if top_k is not None and isinstance(top_k, int):
            token2probs = sorted(token2probs.items(), key=lambda x: x[1], reverse=True )
            token2probs = token2probs[:top_k] 
            token2probs = dict(token2probs)
        anchor_list.append(list(token2probs.keys())) 
        anchor_scores.append(token2probs) 

        # print("-"*60)
    return anchor_list if not return_probs  else pd.Series((anchor_list,anchor_scores))


def filter_anchors_with_probs(df, outputs,  scorer, anchor_col='anchor_lsp_sap', filter=True, return_probs=True, top_k=10, add_wordnet_path_score=False,  add_cpt_score=False, cpt_unmasker=None, mask_string=None, cpt_only=False):
    if not return_probs:
        df['subj_anchors'] = filter_outputs_with_probs(df.sub_label.to_list(), outputs[anchor_col], filter=filter, return_probs=return_probs, top_k=top_k, scorer=scorer)
        # df['obj_anchors'] = filter_outputs_with_probs(df.sub_label.to_list(), outputs['masked_sentences_obj_anchor'], return_probs=return_probs, top_k=top_k)
        # df['obj_mask_sentence'] = filter_outputs_with_probs(df.sub_label.to_list(), outputs['masked_sentences'], return_probs=return_probs, top_k=top_k, scorer=scorer) #remove this to 
        # df['obj_mask_sap'] = filter_outputs_with_probs(df.sub_label.to_list(), outputs['masked_sap'], return_probs=return_probs, top_k=top_k, scorer=scorer)
        # df['top1_subj_anchor'] = df['subj_anchors'].apply(lambda x: x[0])
        # df['top1_incontext_obj'] = df['obj_mask_sentence'].apply(lambda x: x[0])
        return df  
    else:
        df[['subj_anchors', 'subj_anchors_score']] = filter_outputs_with_probs(df.sub_label.to_list(), 
                                                                                outputs[anchor_col], 
                                                                                filter=filter,
                                                                                top_k=top_k, 
                                                                                return_probs=return_probs, 
                                                                                scorer=scorer, add_wordnet_path_score=add_wordnet_path_score, 
                                                                                add_cpt_score=add_cpt_score, cpt_unmasker=cpt_unmasker, mask_string=mask_string, cpt_only= cpt_only
                                                                                )
        # df[['obj_anchors', 'obj_anchors_score']] = filter_outputs_with_probs(df.sub_label.to_list(), outputs['masked_sentences_obj_anchor'], return_probs=return_probs)
        # df[['obj_mask_sentence', 'obj_mask_sentence_score']] = filter_outputs_with_probs(df.sub_label.to_list(), outputs['masked_sentences'], return_probs=return_probs, top_k=top_k, scorer=scorer)
        # df['top1_subj_anchor'] = df['subj_anchors'].apply(lambda x: x[0])
        # df['top1_incontext_obj'] = df['obj_mask_sentence'].apply(lambda x: x[0])
        return df

"""### DAP"""

# def unify_mask(model, text, inp_mask_token, out_mask_token):
#     '''
#     text: the input to be transfered 
#         bert: out_mask_token = [MASK]
#         roberta: out_mask_token = <mask> 
#     '''
#     output = text.replace(inp_mask_token, out_mask_token)
#     return output


def insert_multiple_anchors(original_prompt_source, original_prompts, anchors, sub_position_start, sub_label,  use_original_prompt, incorporate_operation ):
    '''
    original_prompt_source: including the original context, mannually crafted templates, 
    original_prompt: the original prompt, 
    anchors: a list of anchors
    sub_position: 

    return:
        a list of prompts with incorporated anchors
    '''
   

    #print(original_prompt)
    #print(sub_position_start, sub_position_end, type(sub_position_start), type(sub_position_end))
    #print(sub_label, sub_label_string)

    anchored_prompts = []
    if isinstance(original_prompts, list):
        for original_prompt in original_prompts:
            # original_prompt = original_prompt[0]
            sub_position_end = int(sub_position_start) + len(sub_label)
            sub_label_string = original_prompt[sub_position_start : sub_position_end ]

            if use_original_prompt:
                anchored_prompts.append(original_prompt)

            # def insert_anchors_with_or(original_prompt, anchors, sub_label_string, anchors ):
            if incorporate_operation == 'concate_or_single':
                for anchor in anchors:
                    anchored_string = f"{sub_label_string} or {anchor}"
                    anchor_prompt  =  original_prompt[:sub_position_start] + anchored_string + original_prompt[sub_position_end:]
                    anchored_prompts.append(anchor_prompt)
            elif incorporate_operation == 'concate_comma_multiple':
                    anchored_string = "{}, {}".format(sub_label_string, ",".join(anchors))
                    anchor_prompt  =  original_prompt[:sub_position_start] + anchored_string + original_prompt[sub_position_end:]
                    anchored_prompts.append(anchor_prompt)
            elif incorporate_operation == 'replace':
                for anchor in anchors:
                    anchored_string = f"{anchor}"
                    anchor_prompt  =  original_prompt[:sub_position_start] + anchored_string + original_prompt[sub_position_end:]
                    anchored_prompts.append(anchor_prompt)
    return anchored_prompts

def insert_multiple_anchors_by_relacement(original_prompts, anchors, original_string):
    '''
    replace the placeholder [Z] with true values
    '''
    anchored_prompts = []
    for original_prompt in original_prompts:
        for anchor in anchors: 
            anchored_prompts.append(original_prompt.replace(original_string, anchor))
    return anchored_prompts


def fill_anchor_into_dap(df, relation, relation_to_template, use_dap, incorporate_operation, original_prompt_source='template_sap', top_k_anchors = 1,  dap_col_name='masked_sentences_with_subj_anchor',  mask_string = "[MASK]", use_original_prompt=True, ):
    if use_dap:
        if original_prompt_source == 'template_sap':
            template_sap = relation_to_template[relation]['template_sap']
            template_dap = relation_to_template[relation]['template_dap'] 
            df[dap_col_name] = df[['template_sap', 'subj_anchors', 'sub_position','sub_label']].apply(lambda x: insert_multiple_anchors('template_sap', *x) if x[1][0]!='MISSING' else template_sap.replace("[X]", x[0]), axis=1)

        elif original_prompt_source=='masked_sentences':
                df[dap_col_name] = df[['masked_sentences','subj_anchors', 'sub_position', 'sub_label']].apply(lambda x: insert_multiple_anchors(original_prompt_source, x[0], anchors=x[1], sub_position_start=x[2], sub_label=x[3], use_original_prompt=use_original_prompt, incorporate_operation=incorporate_operation ) if x[1][0]!='MISSING' else x[0], axis=1) 

        elif original_prompt_source in ['lsp_dap', 'def_dap']:#, 'masked_sentences']:
            df[dap_col_name] = df[[original_prompt_source,'subj_anchors']].apply(lambda x: insert_multiple_anchors_by_relacement(x[0], anchors=x[1],  original_string='[Z]' ) if x[1][0]!='MISSING' else x[0], axis=1) 

    else: #221005: posy: come back to here when some relaitons are not using dap
        df[dap_col_name] = df[['sub_label', 'subj_anchors']].apply(lambda x: template_sap.replace("[X]", x[:top_k_anchors]), axis=1)
    return df 



''''
args: incorporate_operation ['insert', 'replace', 'concate', ]
output: a list of anchored prompts
'''





def save_dap_templates(df, relation, output_dir):
    # Save DAP to the disk 
    # df_out = df[['pred', 'masked_sentences_with_subj_anchor', 'sub_label', 'obj_label', 'uuid']]
    df_out = df[['masked_sentences_with_subj_anchor', 'sub_label', 'obj_label', 'uuid']]
    df_out = df_out.rename(columns={'masked_sentences_with_subj_anchor': 'masked_sentences'})
    df_out['masked_sentences'] = df_out['masked_sentences'].apply(lambda x: [x])
    # os.makedirs(output_dir, mode=0o777, exist_ok=True)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_path = f'{output_dir}/{relation}.jsonl'
    # json.dump(df_out.to_dict(orient='records'),  output_path) 
    df_out_json = df_out.to_dict( orient='records')

    save_dict_to_json(df_out_json , output_path)
    # {"sub": "abdomen", "obj": "organs", "pred": "HasA", "masked_sentences": ["The abdomen contains [MASK]."], "obj_label": "organs", "uuid": "767f3c6c02e42a55f8b1c314f7167dab"}
    # return df_out

# df[['sub_label', 'masked_sentences', 'obj_mask_sentence', 'obj_label']].sample(50)
# .to_csv("IsA.sample50.filled.human_verification.incontext.csv") #, 'masked_sentences', 'obj_mask_sentence']].sample(50)

"""# Analysis

## evaluation p@k
"""


from evaluation import mean_average_precision, average_precision_at_k, precision_at_k, recall_at_k

def get_precision_at_k_concept(df, relation, pred_cols, label_col, k_list, pred_col_suffix='obj_mask_'):
    '''
    evalaute model predictions in concept level, ignoring the morphology affects (singular, plural)
    '''

    p_at_x = [] #defaultdict() 
    for pred_col in pred_cols: 
        suffix = pred_col.replace(pred_col_suffix, "")
        prec_cur = defaultdict()
        prec_cur['mask_type'] = suffix
        for k in k_list: 
            df[f'p{k}_{suffix}'] = df[[label_col, pred_col]].apply(lambda x: concept_evaluation(x[0], eval(x[1])[:k] if isinstance(x[1], str) else x[1][:k]), axis=1 )
            prec_cur[f'p@{k}'] = round(df[f'p{k}_{suffix}'].mean() , 3)*100

        p_at_x.append(prec_cur)  

    # aggregate the average precision across k 
    df_res = pd.DataFrame(p_at_x) #, columns=['mask_type', 'mAP'])
    df_res['relation'] = [relation]*len(df_res)
    return df_res

def get_precision_at_k(df, relation, pred_cols, label_col, k_list, pred_col_suffix='obj_mask_'):

    p_at_x = [] #defaultdict() 
    for pred_col in pred_cols: 
        suffix = pred_col.replace(pred_col_suffix, "")
        prec_cur = defaultdict()
        prec_cur['mask_type'] = suffix
        for k in k_list: 
            df[f'p{k}_{suffix}'] = df[[label_col, pred_col]].apply(lambda x: 1 if x[0][0] in x[1][:k] else 0, axis=1 )
            prec_cur[f'p@{k}'] = round(df[f'p{k}_{suffix}'].mean() , 3)*100

            # suffix = pred_col.replace("obj_", "")
            # df[f'p@{k}_{suffix}'] = df[['obj_label', pred_col]].apply(lambda x: precision_at_k(y_true=np.array(x[0]), y_pred=np.array(x[1]), k=k), axis=1 )
            # df[f'p@{k}_{suffix}'] = df[['obj_label', pred_col]].apply(lambda x: precision_at_k(y_true=x[0], y_pred=x[1], k=k), axis=1 )

        p_at_x.append(prec_cur)  

    # aggregate the average precision across k 
    df_res = pd.DataFrame(p_at_x) #, columns=['mask_type', 'mAP'])
    df_res['relation'] = [relation]*len(df_res)
    return df_res

def get_highest_mrr_among_labels(label, pred):
    '''
    return the highest rank among the multiple labels. This is applicable to single labels as well, if we the single label is put in a list

    pred: a list of words (candidates)
    label: the true labels, which is a list (different forms of a word, e.g., singular or plurs, like animal and animals)
    '''
    mrr = 0 
    if pred is None: return mrr 

    rank_list = [ pred.index(item) + 1 for item in label if item in pred] 
    if len(rank_list)>0:
        mrr = 1/min(rank_list)

    return mrr 


def get_mrr(df, relation, pred_cols, label_col, pred_col_suffix):
    '''
    mrr is calculated based on the top_k rank, all elements in obj_col are used
    '''
    # def get_mrr_single(label, pred):
    #     '''
    #     pred: a list of words (candidates)
    #     label: the true label 
    #     '''
    #     mrr = 0 
    #     if pred is not None and label in pred:
    #         rank = pred.index(label) + 1
    #         mrr = 1/rank 
    #     return mrr 

    

    mrr = [] 
    for i, pred_col in enumerate(pred_cols):
        cur_mrr = defaultdict()
        suffix = pred_col.replace(pred_col_suffix, "")

        df[f'mrr_{suffix}'] = df[[label_col, pred_col]].apply(lambda x: get_highest_mrr_among_labels(x[0], x[1]), axis=1 ) 
        
        cur_mrr['mask_type'] = suffix
        cur_mrr[f"mrr"] = round(df[f'mrr_{suffix}'].mean(), 3)*100
        mrr.append(cur_mrr)

    mrr_df =  pd.DataFrame(data = mrr) #, columns=['mask_type', 'mrr'])
    # mrr_df['mask_type']= mrr_df['mask_type'].apply(lambda x: x.replace(""))
    mrr_df['relation'] = relation
    return mrr_df 


def get_average_precision(df, relation, true_col, pred_cols, k_list):
    '''
    return the mean average precision for specified pred_cols
    '''
    df['obj_label_list'] = df['obj_label'].apply(lambda x: x if isinstance(x, list) else x)

    mAP_dic = defaultdict()
    for k in k_list:
        key = f'mAP@{k}'
        mAP_dic[key] = defaultdict()
        for pred_col in pred_cols:
            mAP = mean_average_precision(df['obj_label_list'].to_list(), df[pred_col].to_list(), k=k)
            mAP_dic[key][pred_col.replace("obj_", "")] = mAP

    # mAP_df =  pd.DataFrame(data = mAP_dic.items(), columns=['mask_type', 'mAP'])
    mAP_df =  pd.DataFrame(data = mAP_dic) #, columns=['mask_type', 'mAP'])
    mAP_df['relation'] = relation
    mAP_df['mask_type'] = mAP_df.index
    return mAP_df


def get_mean_average_precision(df, relation, true_col, pred_cols, k_list):
    '''
    return the mean average precision for specified pred_cols
    '''
    df['obj_label_list'] = df['obj_label'].apply(lambda x: x if isinstance(x, list) else x)

    mAP_dic = defaultdict()
    for k in k_list:
        key = f'mAP@{k}'
        mAP_dic[key] = defaultdict()
        for pred_col in pred_cols:
            mAP = mean_average_precision(df['obj_label_list'].to_list(), df[pred_col].to_list(), k=k)
            mAP_dic[key][pred_col.replace("obj_", "")] = mAP

    # mAP_df =  pd.DataFrame(data = mAP_dic.items(), columns=['mask_type', 'mAP'])
    mAP_df =  pd.DataFrame(data = mAP_dic) #, columns=['mask_type', 'mAP'])
    mAP_df['relation'] = relation
    mAP_df['mask_type'] = mAP_df.index
    return mAP_df


def get_mean_average_precision_at_k(df, relation, pred_cols, label_col, k_list, pred_col_suffix='obj_mask_'):
    # get the avearage precision per query
    map_at_x = []
    for pred_col in pred_cols: 
        suffix = pred_col.replace(pred_col_suffix, "")
        map_cur = defaultdict()
        map_cur['mask_type'] = suffix

        for k in k_list: 
            df[f'ap{k}_{suffix}'] = df[[label_col, pred_col]].apply(lambda x: average_precision_at_k(y_true=np.array([x[0]]) if isinstance(x[0], str) else np.array(x[0]) , y_pred= np.array(x[1]), k=k), axis=1 )
            map_cur[f'mAP@{k}'] = round(df[f'ap{k}_{suffix}'].mean(), 3)*100
        map_at_x.append(map_cur)

    # aggregate the average precision across k 
    df_res = pd.DataFrame(map_at_x) #, columns=['mask_type', 'mAP'])
    df_res['relation'] = [relation]*len(df_res)
    return df_res



def get_recall_at_k(df, relation, pred_cols, label_col, k_list, pred_col_suffix='obj_mask_'):
    # get the avearage precision per query
    recall_at_x = []
    for pred_col in pred_cols: 
        suffix = pred_col.replace(pred_col_suffix, "")
        recall_cur = defaultdict()
        recall_cur['mask_type'] = suffix

        for k in k_list: 
            df[f'recall_{k}_{suffix}'] = df[[label_col, pred_col]].apply(lambda x: recall_at_k(y_true=np.array([x[0]]) if isinstance(x[0], str) else np.array(x[0]) , y_pred= np.array(x[1]), k=k), axis=1 )
            recall_cur[f'recall@{k}'] = round(df[f'recall_{k}_{suffix}'].mean() ,3 )*100
        recall_at_x.append(recall_cur)

    # aggregate the average precision across k 
    df_res = pd.DataFrame(recall_at_x) #, columns=['mask_type', 'mAP'])
    df_res['relation'] = [relation]*len(df_res)
    return df_res

"""### Analysis helper functions"""

def get_rel_specific_results(df_res_all, mask_type):
# rel = 'mask_sap'
# rel = 'mask_dap'
    dfc = df_res_all.query(f"mask_type == '{mask_type}'").reset_index(drop=True)
    # round(dfc['p@1'].mean(), 2)
    
    overall = {"p@1":   round(dfc['p@1'].mean(), 1),
                "p@3": round(dfc['p@3'].mean(), 1),
                "p@5": round(dfc['p@5'].mean(), 1),
                "p@10": round(dfc['p@10'].mean(), 1),
                "p@10": round(dfc['p@10'].mean(), 1),
                "mrr": round(dfc['mrr'].mean(), 1),
                "relation": 'Overall'
                # "mrr":  round(dfc['mrr'].mean(), 1),
                # "support": dfc['support'].sum() 
                } 
    
    dfc_overall = pd.DataFrame(overall, index=['Overall'])
    # dfc = dfc.sort_values(by=['p@1'])
    dfc = dfc.sort_values(by=['relation'])

    dfc = pd.concat([dfc, dfc_overall]).reset_index(drop=True).sort_values(['relation'])
    
    return  dfc[['relation', 'p@1','p@10', 'mrr', 'p@3', 'p@5', 'mask_type']]

def calculate_gains(df1, df2):
    '''
    df1: sap
    df2: dap
    '''
    metrics = ['p@1', 'p@3', 'p@5', 'p@10', 'mrr']
    df_gains = []
    for col in metrics:
        gains = df2[col] - df1[col]
        gains.column = col 
        df_gains.append(gains)
    
    df_gains = pd.concat(df_gains, axis=1 )
    df_gains['relation'] = df2.relation
    return df_gains[['relation', 'p@1', 'p@10', 'mrr', 'p@3', 'p@5']]

def display_gains(col1, col2, df_res_all, output_file=None):
    ''' 
    goal: diaplay the gains from col1 to col2
    col1: the baseline column 
    '''
    df1 = get_rel_specific_results(df_res_all, mask_type=col1)
    df2 = get_rel_specific_results(df_res_all, mask_type=col2)
    print(f'gains: {col2} - {col1}')
    df_gains = calculate_gains(df1, df2)
    df_gains['mask_type'] =  [ 'Gains(DAP-SAP)'] * len(df_gains.index)
    
    df_out1 = pd.concat([df1, df2, df_gains], axis=1)
    display(df_out1)
    display(df1)
    display(df2)
    if output_file is not None:
        print(f"Saving final results to {output_file}")
        df_out1.to_csv(f"{output_file}")

def aggregate_candidates(dic1, dic2, top_k):
    '''
    input are two list of candidates [{token: score}]
    goal: merge the two lists 
    1. co-occurred words 
        strategy 1 - average: (score1 + score)/2 
        strategy 2 - accumulate: score1 + score
    2. cut off the number of all candiates to top_k
    '''
    keys = set(dic1.keys()).union(dic2.keys())
    new_dic = defaultdict() 

    for key in keys:
        new_dic[key] = dic1.get(key, 0) + dic2.get(key, 0)
    new_dic = dict(Counter(new_dic).most_common(top_k))
    return list(new_dic.keys())




def get_wordnet_avg_path_between_sub_and_anchors(df, oov_path_len = 100):
    '''
    evalaute the anchor quality by measuring
    (1) the average paths between sub_labels and their anchors 
    (2) the coverage of anchors 
    '''
    df['anchor_wordnet_path_len']  = df[['sub_label', 'subj_anchors']].apply(lambda x: [(subj_anchor,get_wordnet_shortest_path_length_between(x[0], subj_anchor, oov_path_len=oov_path_len)) for subj_anchor in x[1]] , axis=1)

    # wn_lemmas = set(wn.all_lemma_names())
    path_len_all = []
    count_oov = 0
    count = 0
    
    for path_lens in df['anchor_wordnet_path_len'].to_list():
        for anchor, path_len in path_lens: 
            count +=1
            if path_len!=oov_path_len:
                path_len_all.append(path_len) 
            else:
                count_oov = count_oov + 1

    avearage_path_len = sum(path_len_all)/len(path_len_all)
    coverage = 1 - count_oov/count

    return round(avearage_path_len, 2) , round(coverage, 3)

def load_config(config_file):
    with open(config_file) as f :
        config = yaml.load(f, Loader=yaml.FullLoader)
    return config 

def concept_evaluation(label, pred):
    '''
    
    label: a list with the singualr and plural labels (e.g., ['tool', 'tools'])
    pred: the top K prediction list 

    return:
        1 if label share with pred else 0  
    '''
    if not isinstance(label, list):
        # label = eval(label)
        label = [label]
        
    if not isinstance(pred, list):
        pred = eval(pred)

    shared = set(label).intersection(set(pred))
    return 1 if len(shared)>0 else 0 
    

def group_evaluation(label, pred):
    '''
    
    label: a list with the singualr and plural labels (e.g., ['tool', 'tools'])
    pred: the top K prediction list 

    return:
        1 if label share with pred else 0  
    '''
    if not isinstance(label, list):
        # label = eval(label)
        label = [label]
        
    if not isinstance(pred, list):
        pred = eval(pred)

    shared = set(label).intersection(set(pred))
    return 1 if len(shared)>0 else 0 
    

def merge_predictions_in_concept_level(uniform_funcion, words, top_k=None ):
    '''
    uniform_function: either signualarize or pluralize 
    '''
    words_uniformed = [uniform_funcion(word) for word in words]
    concepts = list(OrderedDict.fromkeys(words_uniformed))
    return concepts[:top_k] if top_k is not None else concepts





## Main

In [ ]:
# ----------------------------------------  MAIN ------------------------------------ 

#config 
scorer_target_1_prompt =  'probSum'
scorer_target_N_prompts =  'probAvg' #'probLogSum'


# define model 
# vocab = ['flower', 'tool', 'building', 'insect', 'vegetable', 'tree', 'vehicle', 'bird', 'fish', 'trees', 'flowers', 'tools', 'vegetables', 'insects', 'birds', 'vehicles', 'fish', 'buildings' ]
# vocab_sg = ['flower', 'tool', 'building', 'insect', 'vegetable', 'tree', 'vehicle', 'bird', 'fish']
# vocab_pl = ['trees', 'flowers', 'tools', 'vegetables', 'insects', 'birds', 'vehicles', 'fish', 'buildings' ]

device = 0
model= 'bert-large-uncased'
unmasker_sg = get_unmasker(model, device=device) #, targets=vocab)  
unmasker_pl = get_unmasker(model, device=device) #, targets=vocab) 
top_k = 10
batch_size = 100
return_probs=True
data_dir = 'data/hypernymsuite/SHWARTZ/consistency_group/' #sys.argv[1]
debug = True # eval(sys.argv[2])

print(f"Debug: {debug}")
# process data 
# files = [ 'IsA.lsp_sap.jsonl',  'IsA.lsp_dap.jsonl' ] #'IsA.def_sap.jsonl',  'IsA.def_dap.jsonl', 
files = [ 'IsA.lsp_dap.jsonl' ] #'IsA.def_sap.jsonl',  'IsA.def_dap.jsonl', 
# data_dir = 'data/lm_diagnostic_extended/consistency_group/'
save_dir = f'log/{model}/{data_dir.split("/")[-3]}/{data_dir.split("/")[-2]}' 
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

pattern_num = 6

for file in files: 
    filepath = data_dir + file
    df = load_data(filepath, clean_test=True, tokenize=False)
    
    total_row = len(df.index) #total_row = 12994
    chunk_size = 100
    index_list = [i for i in range(0, total_row, chunk_size)]
    if index_list[-1] < total_row-1:
        index_list.append(total_row-1)

    for i, index in enumerate(index_list):
        if i < len(index_list)-1:
            dfi = df.iloc[index : index_list[i+1], :]
            print(index, index_list[i+1], len(dfi.index))
            mask_type = file.split('.')[1]
            if debug: df =  df.head(20)

            #Fill the blank 
            outputs = defaultdict()
            for i in range(1, pattern_num+1):
                # outputs[f'mask_sentences_sg_{i}'] = unmasker_sg(df[f'mask_sentences_singular_{i}'].to_list(), top_k=top_k, batch_size=batch_size) 
                # outputs[f'mask_sentences_pl_{i}'] = unmasker_pl(df[f'mask_sentences_plural_{i}'].to_list(), top_k=top_k, batch_size=batch_size) 

                outputs[f'mask_sentences_sg_{i}'] = [unmasker_sg(x, top_k=2*top_k) for x in df[f'mask_sentences_singular_{i}'].to_list() ]
                outputs[f'mask_sentences_pl_{i}'] = [unmasker_pl(x, top_k=2*top_k) for x in df[f'mask_sentences_plural_{i}'].to_list()] #, batch_size=batch_size) 

                df[[f'obj_mask_sentence_sg_{i}', f'obj_mask_sentence_sg_{i}_score']] = filter_outputs_with_probs(df.sub_label_singular.to_list(), 
                                                                                                    outputs[f'mask_sentences_sg_{i}'], 
                                                                                                    return_probs=return_probs, 
                                                                                                    top_k=2*top_k, 
                                                                                                    scorer=scorer_target_N_prompts,
                                                                                                    filter_inputs = True 
                                                                                                    )

                df[[f'obj_mask_sentence_pl_{i}', f'obj_mask_sentence_{i}_pl_score']] = filter_outputs_with_probs(df.sub_label_plural.to_list(), 
                                                                                                    outputs[f'mask_sentences_pl_{i}'], 
                                                                                                    return_probs=return_probs, 
                                                                                                    top_k=2*top_k, 
                                                                                                    scorer=scorer_target_N_prompts,
                                                                                                    filter_inputs = True 
                                                                                                    )

                df[f'obj_mask_sentence_sg_{i}'] = df[f'obj_mask_sentence_sg_{i}'].apply(lambda x: merge_predictions_in_concept_level( singularize, x, top_k))
                df[f'obj_mask_sentence_pl_{i}'] = df[f'obj_mask_sentence_pl_{i}'].apply(lambda x: merge_predictions_in_concept_level( pluralize, x , top_k))

            df['obj_label_singular'] = df['obj_label_singular'].apply(lambda x: singularize(x) if isinstance(x, str) else singularize(x[0]))
            df['obj_label_plural'] = df['obj_label_plural'].apply(lambda x: pluralize(x) if isinstance(x, str) else pluralize(x[0]))

            # evalaute the predictions 

            ################################################# Evaluation: obj_label ################################################# 
            print("-"*40,"obj_label evaluation", "-"*40)
            pred_col_sg=[x for x in df.columns if 'obj_mask_sentence_sg_' in x and 'score' not in x]
            pred_col_pl=[x for x in df.columns if 'obj_mask_sentence_pl_' in x and 'score' not in x]
            print("pred_col_sg", pred_col_sg)
            print("pred_col_pl", pred_col_pl)

            df_res = []
            for k in [1,10]:
                # for i, (pred_col_sg, pred_col_pl) in enumerate(zip(pred_col_sg, pred_col_pl), start=1):
                for i in range(1, pattern_num):
                    df[f'p1_sg_{i}'] = df[['obj_label_singular', f'obj_mask_sentence_sg_{i}']].apply(lambda x: concept_evaluation(x[0], x[1][:k]), axis=1 )
                    df[f'p1_pl_{i}'] = df[['obj_label_plural', f'obj_mask_sentence_pl_{i}']].apply(lambda x: concept_evaluation(x[0], x[1][:k]), axis=1 )

                pred_col_sg_p1 = [x for x in df.columns if 'p1_sg_' in x ]
                pred_col_pl_p1 = [x for x in df.columns if 'p1_pl_' in x ]


                df['p1_sg'] = df[pred_col_sg_p1].apply(lambda x: int(all(ele == 1 for ele in x)), axis=1)
                df['p1_pl'] = df[pred_col_pl_p1].apply(lambda x: int(all(ele == 1 for ele in x)), axis=1)

                df['p1_sgpl'] = df[['p1_sg', 'p1_pl']].apply(lambda x: 1 if x[0]==1 and x[1]==1 else 0, axis=1)

                acc_sg = round(df['p1_sg'].sum()/len(df.index), 3) * 100
                acc_pl = round(df['p1_pl'].sum()/len(df.index), 3) * 100
                acc_sgpl = round(df['p1_sgpl'].sum()/len(df.index), 3) * 100

                df_res.append({"K":k, 'Singular': acc_sg, 'Plural': acc_pl, 'Paired Singular-Plural': acc_sgpl})

            df_res = pd.DataFrame(data=df_res)
            df_res['mask_type'] = mask_type
            df_res['data_dir'] = data_dir 

            display(df_res)
            outpath = f'{save_dir}/{file.replace("jsonl", "csv")}'
            # df.to_csv(outpath)

            res_outpath = f'{save_dir}/{file.replace("jsonl", "tsv")}'
        #     df_res.to_csv(res_outpath)

            print(f"Save {outpath}")
            print(f"Save {res_outpath}")
            print("")


20

In [16]:
# df.head()
# splitting dataframe by row index
total_row = len(df.index) #total_row = 12994
chunk_size = 3000
index_list = [i for i in range(0, total_row, chunk_size)]
if index_list[-1] < total_row-1:
    index_list.append(total_row-1)

for i, index in enumerate(index_list):
    if i < len(index_list)-1:
#         print(index, index_list[i+1])
        dfi = df.iloc[index : index_list[i+1], :]
    
# df_1 = df.iloc[:1000,:]
# df_2 = df.iloc[1000:,:]
# df_1.head()


0 3000
3000 6000
6000 9000
9000 12000
12000 12993
